In [2]:
import cv2
import numpy as np

roi_hist_save = np.array([[  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [255.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.]], dtype=np.float32)

In [19]:
import cv2
import numpy as np

def run_ex_cam(num, x, y, size, roi_hist):
    cap = cv2.VideoCapture('../vid/swarm_drone/swarm_drone{}.mp4'.format(num))
    orb = cv2.ORB_create(nfeatures=1000)
    #orb = cv2.ORB_create()
    #cap.set(cv2.CAP_PROP_POS_FRAMES, 1000)
    ret, frame = cap.read()
    scale = 1
    frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
    whole = size // scale
    y, x = y // scale, x // scale
    crop = frame[y:y + whole, x:x + whole]
    r, h, c, w = y, whole, x, whole
    #r,h,c,w = 250,90,400,125 
    track_window = (c,r,w,h)
    
    kp_prev, des_prev = orb.detectAndCompute(crop,None)
    height,width, depth = crop.shape
    black = np.zeros((height,width, depth), np.uint8)
    for sing_kp in kp_prev:
        x, y = sing_kp.pt
        cv2.circle(black,(np.int(x),np.int(y)), 5, (0,255,0), -1)
    cv2.imshow('crop', crop)
    cv2.imshow('black', black)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
    
    i = 0
    while True:
        ret, frame = cap.read()
        if ret == False or cv2.waitKey(1) & 0xff == 27:
            break
        copy = frame.copy()
        kp, des = orb.detectAndCompute(frame,None)
        frame = cv2.drawKeypoints(frame,kp,None,color=(0,255,0), flags=0)
        height,width, depth = frame.shape
        black = np.zeros((height,width, depth), np.uint8)
        for sing_kp in kp:
            x, y = sing_kp.pt
            cv2.circle(black,(np.int(x),np.int(y)), 5, (0,255,0), -1)
        black = cv2.morphologyEx(black, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))
        #cv2.imshow('frame', frame)
        cv2.imshow('black', black)
        hsv = cv2.cvtColor(black, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        # apply meanshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        # Draw it on image
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, 255,2)
        
        tot_x, tot_y = 0, 0
        num = len(pts)
        for cur_x, cur_y in pts:
            tot_x += cur_x
            tot_y += cur_y
        px = tot_x // num
        py = tot_y // num
        
        out = copy[py - whole // 2:py + whole // 2, px - whole // 2:px + whole // 2]
        cv2.imshow('out', out)
        cv2.imshow('img2',img2)
        i += 1
    cv2.destroyAllWindows()
    cap.release()
#1
run_ex_cam(num='', x=550, y=230, size=100, roi_hist=roi_hist_save)

error: C:\bld\opencv_1506447021968\work\opencv-3.3.0\modules\highgui\src\window.cpp:325: error: (-215) size.width>0 && size.height>0 in function cv::imshow


In [42]:
cap = cv2.VideoCapture('../vid/swarm_drone/swarm_drone.mp4')
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
i = 0
while True:
    ret, frame = cap.read()
    if ret == False or cv2.waitKey(1) & 0xff == 27:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    if p1 is None or i % 100 == 0:
        p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    i += 1

cv2.destroyAllWindows()
cap.release()

In [39]:
type(None)

NoneType